In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
device

'cuda'

In [31]:
data_path = '../data/tiny_shakespeare.txt'
eval_iters = 200

In [4]:
with open(data_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print(''.join(chars))
print(VOCAB_SIZE)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda v: ''.join([itos[i] for i in v])

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
BLOCK_SIZE = 8

In [9]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]
for t in range(BLOCK_SIZE):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} the target is: {target}')

When input is tensor([18]) the target is: 47
When input is tensor([18, 47]) the target is: 56
When input is tensor([18, 47, 56]) the target is: 57
When input is tensor([18, 47, 56, 57]) the target is: 58
When input is tensor([18, 47, 56, 57, 58]) the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [10]:
torch.manual_seed(1337)
BATCH_SIZE = 4
BLOCK_SIZE = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('----')

for b in range(BATCH_SIZE):
    for t in range(BLOCK_SIZE):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context.tolist()} the target is: {target}')

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='cuda:0')
----
When input is [24] the target is: 43
When input is [24, 43] the target is: 58
When input is [24, 43, 58] the target is: 5
When input is [24, 43, 58, 5] the target is: 57
When input is [24, 43, 58, 5, 57] the target is: 1
When input is [24, 43, 58, 5, 57, 1] the target is: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
When input is [44] the target is: 53
When input is [44, 53] the target is: 56
When input is [44, 53, 56] the target is: 1
When input is [44, 53, 56, 1] the target is: 5

In [32]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [23]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vs):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vs, vs)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            b, t, c = logits.shape        
            logits = logits.view(b*t, c)
            targets = targets.view(b*t)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx = idx.to(device)
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel(VOCAB_SIZE)
m = model.to(device)
logits, loss = m(xb, yb)
print(logits.shape, loss)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) tensor(4.1519, device='cuda:0', grad_fn=<NllLossBackward>)

fzDbRf bnNdc.wd,fTkO.lT,xCtK
K:xPjJabBbeA$3etaSvgO-33SM-??ga3d
M!FYVXHFbXfzuhqcOC$v.tbfFYdXE!FvaAetj


In [28]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
epochs = 10000

In [39]:
for epoch in range(epochs):
    if epoch % 1000 == 0:
        losses = estimate_loss()
        print(f'step {epoch}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}')
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step 0: train loss 2.4994, val loss 2.5379
step 1000: train loss 2.5029, val loss 2.5076
step 2000: train loss 2.5057, val loss 2.4909
step 3000: train loss 2.4558, val loss 2.5077
step 4000: train loss 2.5065, val loss 2.4913
step 5000: train loss 2.4983, val loss 2.5090
step 6000: train loss 2.4683, val loss 2.4951
step 7000: train loss 2.4771, val loss 2.4950
step 8000: train loss 2.4615, val loss 2.4885
step 9000: train loss 2.4539, val loss 2.4904
2.269038200378418


In [40]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=300)[0].tolist()))


I tano iru fo. floiroret HEnk;
CUCaden tck is, d cer t ftanofallon bay ho s, agallen, meseveminds s; te worimyoin ie--tofit thike a wowhedichea blare gofably senInirstha theint de mas, the an be ke imyol menat m inor woeing brwimaviselllawe e wo IICis;
ST:
GRDo y'sill! whertaperulllalop!grthe olirco
